# Data exploring tools
A recap of the pure python tools designed to explore the "Irish" dataset during the Hackathon

In [2]:
import sys, os, csv, requests, re, bs4, datetime, time, threading, wget, pdfminer, geopy #geopandas, 
import pandas as pd
import matplotlib.pyplot as plt
from pdfminer.high_level import extract_text
from geopy.geocoders import Nominatim
import numpy as np
from requests import get
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.feature_extraction.text import CountVectorizer 
import json
from nltk.corpus import stopwords

In [3]:
path='C:/Users/etien/Python/Ireland'
os.chdir(path)
os.getcwd()

'C:\\Users\\etien\\Python\\Ireland'

## Here is a view of what my workspace look like, after scraping PDFs (Reports folder) and converting the PDF to txt file (Reports/Plain text folder):

In [4]:
print('*** in the working directory:\n')
print(os.listdir())
print('\n*** in the Reports directory:')
print('\nhead: ',os.listdir('Reports/')[0:3], '...')
print('... tail: ', os.listdir('Reports/')[-3:])
print('there are:', len(os.listdir('Reports/')[:-1]), 'files')
print('path in PATH_REPORTS')
print('filenames in LIST_REPORTS')
PATH_REPORTS='Reports/'
LIST_REPORTS=os.listdir('Reports/')[:-1]

print('\n\n*** in the Reports/Plain text directory:')
print('\nhead: ',os.listdir('Reports/Plain text/')[0:3], '...')
print('... tail: ', os.listdir('Reports/Plain text/')[-3:])
print('there are:', len(os.listdir('Reports/Plain text/')[:-1]), 'files')
print('path in PATH_PLAIN')
print('filenames in LIST_PLAIN')
PATH_PLAIN='Reports/Plain text/'
LIST_PLAIN=os.listdir('Reports/Plain text')[:-2]

*** in the working directory:

['data_to_visualize.json', 'data_to_visualize0.json', 'data_to_visualize1.json', 'Ireland_Map.gpkg', 'Reports', 'words_alpha.txt']

*** in the Reports directory:

head:  ['001125B_24_11_2011.pdf', '00467B_15_12_2011.pdf', '00512D_09_06_2011.pdf'] ...
... tail:  ['91550B_13_03_2018.pdf', '91614B_02_12_2020.pdf', 'Plain text']
there are: 2470 files
path in PATH_REPORTS
filenames in LIST_REPORTS


*** in the Reports/Plain text directory:

head:  ['00467B_15_12_2011.txt', '00538V_25_10_2012.txt', '00606M_25_05_2015.txt'] ...
... tail:  ['91614B_02_12_2020.txt', 'filtered', 'report1_19557H_04_12_2014.txt']
there are: 2010 files
path in PATH_PLAIN
filenames in LIST_PLAIN


# Counting words across the corpus

In [5]:
total_words=set()
path=PATH_PLAIN
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN+fname,'r')
    for line in f.readlines():
        total_words |= set([word.strip().lower() for word in line.split()])
len(total_words)

45598

**45598 total set of words in the corpus. However many are not realwords.**

In [6]:
realwords_in_english=set([word.strip().lower() for word in open('words_alpha.txt','r').readlines()])
len(realwords_in_english)

370103

**English words from https://github.com/dwyl/english-words . Using words_alpha.txt but may be better to use words.txt (has the hyphen-containing words)**

In [7]:
total_realwords=total_words.intersection(realwords_in_english)
len(total_realwords)

10566

**There are 10566 different "real" words in the corpus. Now let's count them across the corpus.**

In [8]:
word_counts={}
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in total_realwords:
                word_counts[word]=word_counts.get(word,0)+1
sorted_word_counts = { k:v for k, v in sorted(word_counts.items(), key=lambda item: item[1], reverse=True) }
sorted_word_counts

{'the': 269542,
 'of': 234138,
 'and': 225463,
 'in': 126015,
 'to': 122345,
 'is': 93015,
 'a': 91111,
 'school': 60382,
 'for': 57805,
 'are': 47997,
 'with': 46542,
 'that': 39991,
 'learning': 37743,
 'very': 34875,
 'be': 33503,
 'on': 30553,
 'good': 26543,
 'should': 25163,
 'pupils': 24355,
 'management': 23609,
 'an': 22337,
 'teaching': 22273,
 'was': 22107,
 'their': 21929,
 'has': 21137,
 'board': 21012,
 'quality': 20247,
 'as': 19759,
 'support': 17797,
 'by': 17154,
 'students': 17088,
 'all': 17029,
 'work': 15148,
 'teachers': 14652,
 'this': 14527,
 'have': 14058,
 'na': 13601,
 'development': 13483,
 'been': 13132,
 'were': 12899,
 'range': 12171,
 'staff': 10989,
 'use': 10701,
 'some': 10230,
 'student': 9986,
 'well': 9923,
 'high': 9809,
 'from': 9532,
 'needs': 9364,
 'at': 9255,
 'planning': 9225,
 'principal': 9120,
 'further': 8927,
 'lessons': 8833,
 'leadership': 8797,
 'which': 8728,
 'effective': 8298,
 'practice': 8229,
 'provided': 8078,
 'there': 8038,

**Let's remove the stopwords from the nltk library**

In [9]:
for stopword in stopwords.words('english'):
    if stopword in sorted_word_counts:
        del sorted_word_counts[stopword]
sorted_word_counts

{'school': 60382,
 'learning': 37743,
 'good': 26543,
 'pupils': 24355,
 'management': 23609,
 'teaching': 22273,
 'board': 21012,
 'quality': 20247,
 'support': 17797,
 'students': 17088,
 'work': 15148,
 'teachers': 14652,
 'na': 13601,
 'development': 13483,
 'range': 12171,
 'staff': 10989,
 'use': 10701,
 'student': 9986,
 'well': 9923,
 'high': 9809,
 'needs': 9364,
 'planning': 9225,
 'principal': 9120,
 'lessons': 8833,
 'leadership': 8797,
 'effective': 8298,
 'practice': 8229,
 'provided': 8078,
 'parents': 7933,
 'provision': 7818,
 'recommendations': 7812,
 'report': 7547,
 'areas': 7459,
 'team': 7402,
 'also': 7173,
 'ar': 6957,
 'subject': 6780,
 'recommended': 6706,
 'review': 6681,
 'ensure': 6669,
 'assessment': 6636,
 'used': 6572,
 'positive': 6490,
 'senior': 6327,
 'number': 6276,
 'improvement': 6264,
 'pupil': 6230,
 'language': 6041,
 'progress': 5991,
 'inspection': 5802,
 'skills': 5716,
 'overall': 5450,
 'plan': 5413,
 'class': 5354,
 'approach': 5213,
 'go

# Creating topic-related collections
## Using an API to create a topic-related set of words

Example shown : the sports-related word collection

_Datamuse_ allows us to find new words related to our topic


In [10]:
topic = ['sports','football','hockey']
topic_related_words=set()
for word in topic:
    url = 'https://api.datamuse.com/words?topics='+word
    for element in get(url).json():
        if element['word'] not in stopwords.words('english'):
            topic_related_words |= {element['word']}
len(topic_related_words)

236

Starting from 3 words, we now have 236 of them !

In [11]:
topic_related_words

{'activities',
 'adventure',
 'alpine skiing',
 'aquatics',
 'association football',
 'athlete',
 'athletes',
 'athletic',
 'athletic game',
 'athletics',
 'badminton',
 'ball',
 'ballgame',
 'balloon',
 'ballplayer',
 'bantams',
 'baseball',
 'basketball',
 'basketball equipment',
 'bball',
 'boast',
 'body',
 'bodycheck',
 'bonspiel',
 'boxing',
 'broomball',
 'bullet',
 'calcium',
 'campaign',
 'campaigns',
 'canadian football',
 'candlepin bowling',
 'cavort',
 'championship',
 'cheerleading',
 'club',
 'coaches',
 'codes',
 'competitions',
 'contact sport',
 'court game',
 'cricket',
 'croix',
 'cujo',
 'cup',
 'curlers',
 'cycling',
 'day game',
 'defenseman',
 'disciplines',
 'disport',
 'entertainment',
 'enthusiast',
 'enthusiasts',
 'equestrian',
 'events',
 'exercise',
 'exercises',
 'faceoff',
 'famous',
 'farm team',
 'feature',
 'federations',
 'field game',
 'field hockey',
 'fifa',
 'figure skating',
 'flask',
 'foot',
 'football',
 'football coach',
 'football league',

Obviously we don't need all these words. Let's select those that are in the corpus

In [12]:
topic_related_words_in_corpus = topic_related_words.intersection(total_realwords)
len(topic_related_words_in_corpus)

58

**Filtering the collection, we have now 58 words (from the 236 obtained with the Datamuse API enrichment)**
**58 is small enough to clean it manually. Let's remove words that are not specific to sports**

## Manually cleaning the collection
For each word of the set, the user is asked whether we should keep it or remove it

In [12]:
copy = topic_related_words_in_corpus.copy()
for word in list(copy):
    if input('Keep '+ word + '? Press Enter to keep. Press any key + Enter to remove : '):
        topic_related_words_in_corpus.remove(word)

Keep campaign? Press Enter to keep. Press any key + Enter to remove : n
Keep athletic? Press Enter to keep. Press any key + Enter to remove : 
Keep recreation? Press Enter to keep. Press any key + Enter to remove : n
Keep campaigns? Press Enter to keep. Press any key + Enter to remove : n
Keep athletics? Press Enter to keep. Press any key + Enter to remove : 
Keep famous? Press Enter to keep. Press any key + Enter to remove : n
Keep player? Press Enter to keep. Press any key + Enter to remove : n
Keep handball? Press Enter to keep. Press any key + Enter to remove : 
Keep coaches? Press Enter to keep. Press any key + Enter to remove : n
Keep foot? Press Enter to keep. Press any key + Enter to remove : n
Keep games? Press Enter to keep. Press any key + Enter to remove : n
Keep match? Press Enter to keep. Press any key + Enter to remove : 
Keep body? Press Enter to keep. Press any key + Enter to remove : n
Keep rugby? Press Enter to keep. Press any key + Enter to remove : 
Keep sports? Pr

In [13]:
topic_related_words_in_corpus

{'aquatics',
 'athletic',
 'athletics',
 'badminton',
 'ball',
 'basketball',
 'competitions',
 'cricket',
 'cycling',
 'equestrian',
 'football',
 'golf',
 'gym',
 'gymnastics',
 'handball',
 'hockey',
 'match',
 'physical',
 'rugby',
 'soccer',
 'sport',
 'sports',
 'swimming',
 'tennis'}

**This collection looks more specific now!**
Here are the words we removed

In [14]:
removed = copy - topic_related_words_in_corpus
removed

{'activities',
 'adventure',
 'body',
 'bullet',
 'campaign',
 'campaigns',
 'club',
 'coaches',
 'codes',
 'cup',
 'disciplines',
 'events',
 'exercise',
 'exercises',
 'famous',
 'feature',
 'foot',
 'fun',
 'game',
 'games',
 'ice',
 'league',
 'leisure',
 'mathematical',
 'movement',
 'play',
 'player',
 'players',
 'playing',
 'recreation',
 'recreational',
 'stick',
 'team',
 'teams'}

In [14]:
copy = topic_related_words_in_corpus.copy()
removed = {'activities',
 'adventure',
 'body',
 'bullet',
 'campaign',
 'campaigns',
 'club',
 'coaches',
 'codes',
 'cup',
 'disciplines',
 'events',
 'exercise',
 'exercises',
 'famous',
 'feature',
 'foot',
 'fun',
 'game',
 'games',
 'ice',
 'league',
 'leisure',
 'mathematical',
 'movement',
 'play',
 'player',
 'players',
 'playing',
 'recreation',
 'recreational',
 'stick',
 'team',
 'teams'}
topic_related_words_in_corpus = copy-removed

In [15]:
print('we removed manually ', len(removed), ' words and we kept', len(topic_related_words_in_corpus),' words.')

we removed manually  34  words and we kept 24  words.


**Let's count these words across the corpus now !**

In [16]:
topic_related_word_counts={}
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in topic_related_words_in_corpus:
                topic_related_word_counts[word]=topic_related_word_counts.get(word,0)+1
sorted_topic_related_word_counts = { k:v for k, v in sorted(topic_related_word_counts.items(), key=lambda item: item[1], reverse=True) }
sorted_topic_related_word_counts

{'physical': 519,
 'sports': 299,
 'competitions': 82,
 'match': 81,
 'sport': 56,
 'swimming': 38,
 'football': 21,
 'ball': 20,
 'basketball': 16,
 'rugby': 12,
 'gym': 12,
 'soccer': 10,
 'athletics': 9,
 'gymnastics': 9,
 'tennis': 8,
 'hockey': 6,
 'handball': 2,
 'cycling': 1,
 'aquatics': 1,
 'cricket': 1,
 'equestrian': 1,
 'badminton': 1}

**Second-thoughts deletion :**

In [17]:
for element in ['physical']:
    if element in sorted_topic_related_word_counts:
        del sorted_topic_related_word_counts[element]

## Making a JSON file with the counts of sports-related words in each report of the corpus

In [18]:
data_to_visualize=[]

In [19]:
data_to_visualize.append({'topic':'sports', 'value_by_school':[]})
for fname in LIST_PLAIN:
    f=open(PATH_PLAIN + fname,'r')
    count=0
    for line in f.readlines():
        for word in [word.strip() for word in line.split()]:
            if word in sorted_topic_related_word_counts:
                count+=1
    data_to_visualize[-1]['value_by_school'].append({'school':fname[:6],'value':count})
data_to_visualize

[{'topic': 'sports',
  'value_by_school': [{'school': '00467B', 'value': 1},
   {'school': '00538V', 'value': 2},
   {'school': '00606M', 'value': 1},
   {'school': '00729F', 'value': 1},
   {'school': '00752A', 'value': 0},
   {'school': '00779U', 'value': 0},
   {'school': '00856M', 'value': 1},
   {'school': '00934G', 'value': 0},
   {'school': '00984V', 'value': 0},
   {'school': '01013N', 'value': 0},
   {'school': '01086R', 'value': 2},
   {'school': '01170G', 'value': 0},
   {'school': '01285A', 'value': 1},
   {'school': '01300Q', 'value': 0},
   {'school': '01356U', 'value': 0},
   {'school': '01421F', 'value': 0},
   {'school': '01572D', 'value': 0},
   {'school': '01574H', 'value': 1},
   {'school': '01607T', 'value': 4},
   {'school': '01676P', 'value': 1},
   {'school': '01687U', 'value': 0},
   {'school': '01795A', 'value': 1},
   {'school': '01840C', 'value': 0},
   {'school': '01866U', 'value': 0},
   {'school': '01867W', 'value': 0},
   {'school': '02114B', 'value': 0}

In [20]:
f=open('data_to_visualize.json','w')
json.dump(data_to_visualize,f)

## There's still another way to enrich our topic-related word set
But before that we need to introduce other tools to _see the words in their context_... So, be patient, we're not finished the with sports-related word collection!

# Searching one word in the corpus
The words dog and dogs appeared just once across the corpus. Let's examing in what files.

In [21]:
stop = False
while not stop:
    word_to_search = input('Search word : ')
    if not word_to_search:
        stop = True
    else:
        for fname in LIST_PLAIN:
            f=open(PATH_PLAIN + fname,'r')
            count=0
            for line in f.readlines():
                for word in [word.strip() for word in line.split()]:
                    if (word == word_to_search):
                        count+=1
            if count:
                print(fname, ' : ', count, ' occurence(s).')

Search word : dog
18408I_05_12_2018.txt  :  1  occurence(s).
Search word : dogs
17730I_26_01_2016.txt  :  1  occurence(s).
Search word : soccer
01607T_15_04_2014.txt  :  1  occurence(s).
16976M_08_01_2020.txt  :  1  occurence(s).
17812K_30_09_2015.txt  :  1  occurence(s).
19520H_10_06_2014.txt  :  1  occurence(s).
19946S_06_02_2019.txt  :  1  occurence(s).
62570R_26_02_2019.txt  :  1  occurence(s).
63290Q_20_10_2015.txt  :  1  occurence(s).
68070E_05_07_2012.txt  :  1  occurence(s).
71123Q_19_09_2017.txt  :  1  occurence(s).
71400Q_26_05_2014.txt  :  1  occurence(s).
Search word : 


# Displaying a word in its context
Context is important ! Let's see why...

In [23]:
def sentences_in_file(fname,word):
    f=open(PATH_PLAIN + fname,'r')
    sentences_list = f.read().split('.')
    sentences_with_word = []
    for sentence in sentences_list:
        if word in sentence:
            sentences_with_word.append(sentence)
    return sentences_with_word

fname=input('File name : ')
word=input('Word : ')
for sentence in sentences_in_file(fname,word):
    print(sentence)

File name : 17730I_26_01_2016.txt
Word : dogs
  The  ongoing  issue  in  relation  to  the  presence  of  dogs  on 
the school grounds must be resolved by the board as a matter of priority


In [24]:
fname=input('File name : ')
word=input('Word : ')
for sentence in sentences_in_file(fname,word):
    print(sentence)

File name : 18408I_05_12_2018.txt
Word : dog
  The effective and flexible use of activities in PE, Art, Music, cooking, along with students’ access to services providing equine therapy, dog therapy and other supports contribute to the supportive, learner-centred culture evident in the school


**Wow ! see how much these two contexts are different ?**

# Displaying all contexts of one word

In [26]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print('quote : ' + sentence)

Word : fun 

In file 13512B_26_09_2016.txt:
quote :  It would also be worthwhile providing an extended range of fun 
opportunities  for  pupils  to  learn  the  language

In file 16044J_04_09_2015.txt:
quote :   There  is  evidence  that 
pupils in the junior room are developing social, language and communication skills in 
a fun and supportive environment

In file 16573N_26_10_2012.txt:
quote :   The 
promotion of a positive, inclusive attitude to sport and the balance of competitive and fun 
activities  are  commended

In file 18033O_20_12_2019.txt:
quote :    The quality of learning experiences provided to the pupils is a particular strength of the school: pupils engage in varied, rich, fun and meaningful activities

In file 18178R_08_05_2015.txt:
quote :   Pupils  derive  benefit  and 
fun from poetry activities and songs

In file 18506I_06_01_2015.txt:
quote :  
•  School reports have been updated in line with circular 56/11 & 45/14 
•  Maths for fun has been implemented and all c

**Note: we entered 'fun ' because 'fun' would retrieve any words containing 'fun' like 'funds' and 'functional'. To avoid this, we should check if (word in sentence.split().strip()) before adding the sentence**

In [86]:
def sentences_in_file(fname,word):
    f=open(PATH_PLAIN + fname,'r')
    sentences_list = f.read().split('.')
    sentences_with_word = []
    for sentence in sentences_list:
        if word in sentence and (word in [word.strip() for word in sentence.split()]):
            sentences_with_word.append(sentence)
    return sentences_with_word

def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print('quote : ' + sentence)

Word : gym

In file 19724A_31_05_2018.txt:
quote :  Pupils also utilise external facilities to access horse riding, swimming, and bowling, and exercise at a local gym

In file 60383I_06_02_2019.txt:
quote :  Facilities are maintained to a high standard and the construction of the ASD unit and the development of the fitness suite in the gym have been welcome additions to the school

In file 60490J_09_10_2018.txt:
quote :  As there is no gym or hall in the building, school management enables access to a local gym for some physical activities
quote :  Provision has since been made for a double timetabled class for 1st 2nd and 3rd years, using local gym facilities, the school yard and local sporting facilities

In file 60890C_11_02_2015.txt:
quote :  
 
The school has established and maintains close links with the local Rathmines community, 
working in collaboration with local schools, businesses, and a range of local organisations 
and services including the library and the local gym faci

**'gym' is retrieved, not 'gymnastics'**

# Creating topic-related collections, continued
**By randomly exploring the context of sport-related words in the corpus, we can enrich greatly the collection :**

In [27]:
def sentences_in_corpus(word):
    sentences_by_file={}
    for fname in LIST_PLAIN:
        sentences_with_word=sentences_in_file(fname,word)
        if sentences_with_word:
            sentences_by_file[fname]=sentences_with_word
    return sentences_by_file

word=input('Word : ')
sentences_in_corpus=sentences_in_corpus(word)
for fname in sentences_in_corpus:
    print('\nIn file ' + fname + ':')
    for sentence in sentences_in_corpus[fname]:
        print(sentence)

Word : athletics

In file 01607T_15_04_2014.txt:
 

 

External  coaches  provide  tuition  in  a  range  of  activities  such  as  athletics,  table  tennis, 
soccer and rugby under the direction of the class teachers

In file 01676P_25_11_2016.txt:
 

 

  The  practice  whereby  parents  are  required  to  pay  external  tutors  for  lessons  in 
gymnastics,  yoga  and  athletics  which  take  place  during  the  school  day  contravenes 
Departmental  policy

In file 04075E_29_05_2015.txt:
  The  pupils’  creativity 
across the curriculum is celebrated through excellent displays of their work and their 
achievements are recognised and applauded at fortnightly assembly Participation in a 
range  of  co-curricular  activities  including  the  school  band,  athletics  and  games 
provides  a  particularly  positive  experience  for  the  pupils

In file 08342V_27_03_2012.txt:
 
Parents  and  teachers  have  co-operated  successfully  in  encouraging  pupils’  interest  in 
Mathematic

**From this knowledge, and other search, we enriched the sports_collection with other sports that appear in the corpus:**

In [28]:
sports_collection=[
 'sports',
 'competitions',
 'sport',
 'swimming',
 'football',
 'ball',
 'basketball',
 'rugby',
 'gym',
 'soccer',
 'athletics',
 'gymnastics',
 'tennis',
 'hockey',
 'handball',
 'cycling',
 'aquatics',
 'cricket',
 'equestrian',
 'badminton',
 'table tennis',
 'yoga',
 'hurling',
 'aquatics',
 'Gaelic games',
 'Gaelic football',
 'tag-rugby','playground games','PE']
sports_collection

['sports',
 'competitions',
 'sport',
 'swimming',
 'football',
 'ball',
 'basketball',
 'rugby',
 'gym',
 'soccer',
 'athletics',
 'gymnastics',
 'tennis',
 'hockey',
 'handball',
 'cycling',
 'aquatics',
 'cricket',
 'equestrian',
 'badminton',
 'table tennis',
 'yoga',
 'hurling',
 'aquatics',
 'Gaelic games',
 'Gaelic football',
 'tag-rugby',
 'playground games',
 'PE']

# Visualizing the use of words from a topic-related collection in the corpus

In [31]:
def data_from_topic(topic, topic_words):
    data=[{'topic': topic, 'value_by_school':[]}]
    for fname in LIST_PLAIN:
        f=open(PATH_PLAIN + fname,'r')
        count=0
        for line in f.readlines():
            for word in [word.strip() for word in line.split()]:
                if word in topic_words :
                    count+=1
        data[0]['value_by_school'].append({'school':fname[:6],'date': fname[7:17],'value':count})
    return data

data = data_from_topic('sports', sports_collection)

[{'topic': 'sports',
  'value_by_school': [{'school': '00467B', 'date': '15_12_2011', 'value': 1},
   {'school': '00538V', 'date': '25_10_2012', 'value': 2},
   {'school': '00606M', 'date': '25_05_2015', 'value': 1},
   {'school': '00729F', 'date': '23_06_2014', 'value': 0},
   {'school': '00752A', 'date': '15_10_2012', 'value': 0},
   {'school': '00779U', 'date': '28_04_2016', 'value': 0},
   {'school': '00856M', 'date': '03_03_2016', 'value': 1},
   {'school': '00934G', 'date': '27_06_2014', 'value': 0},
   {'school': '00984V', 'date': '14_11_2018', 'value': 0},
   {'school': '01013N', 'date': '05_10_2018', 'value': 0},
   {'school': '01086R', 'date': '19_09_2016', 'value': 2},
   {'school': '01170G', 'date': '26_10_2012', 'value': 0},
   {'school': '01285A', 'date': '26_10_2017', 'value': 1},
   {'school': '01300Q', 'date': '08_10_2020', 'value': 0},
   {'school': '01356U', 'date': '29_11_2016', 'value': 0},
   {'school': '01421F', 'date': '16_05_2019', 'value': 0},
   {'school': '0

# Using pandas to examine the sport-related occurrences

**Let's have a clean data frame with the number of occurences of sport-related words for each reports**

In [74]:
data = data_from_topic('sports', sports_collection)
data_df=pd.DataFrame(data[0]['value_by_school'])
def merge_columns(row):
    row.iloc[0] = row.iloc[0] + '_' + row.iloc[1]
    return row
data_df = data_df.apply(merge_columns, axis='columns')
data_df.columns = ['Report Id', 'date', 'sports_value'] #i renamed Report Id and sports_value
data_df = data_df.set_index('Report Id')
data_df = data_df.drop('date', axis=1)
data_df = data_df.sort_index()
data_df

,sports_value
Report Id,
00467B_15_12_2011,1
00538V_25_10_2012,2
00606M_25_05_2015,1
00729F_23_06_2014,0
00752A_15_10_2012,0
...,...
91516B_21_09_2018,0
91517D_24_06_2014,1
91530S_07_11_2014,0


**Let's import the dataframe for the inspection reports (we have it on csv file)**

In [75]:
general_inspectionReports = pd.read_csv('General_inspectionReports.csv')
def merge_columns(row):
    row.iloc[0] = row.iloc[2] + '_' + row.iloc[1]
    return row
general_inspectionReports = general_inspectionReports.apply(merge_columns, axis='columns')
general_inspectionReports.columns = ['Report Id', 'Date', 'School Roll No.', 'County', 'School Name',
       'School Level', 'Inspection Type', 'Subject', 'URL']
general_inspectionReports = general_inspectionReports.set_index('Report Id')
general_inspectionReports = general_inspectionReports.sort_index()
general_inspectionReports

,Date,School Roll No.,County,School Name,School Level,Inspection Type,Subject,URL
Report Id,,,,,,,,
001125B_24_11_2011,24_11_2011,001125B,Leitrim,St Joseph’s National School Leitrim Village ...,Primary,Whole School Evaluation,NaN,report1_01125B.pdf
00467B_15_12_2011,15_12_2011,00467B,Cork,Ballinspittle National School Kinsale Cork,Primary,Whole School Evaluation,NaN,report1_00467B.pdf
00512D_09_06_2011,09_06_2011,00512D,Cork,Scoil Bhríde Presentation Primary School Mid...,Primary,Whole School Evaluation,NaN,report1_00512D.pdf
00538V_25_10_2012,25_10_2012,00538V,Kerry,"Convent Primary School Daingean Uí Chúis, Co. ...",Primary,Whole School Evaluation,NaN,report1_EV_00538V.pdf
00606M_25_05_2015,25_05_2015,00606M,Tipperary N.R.,"Monard National School, Monard, Co Tipperary",Primary,Whole School Evaluation,NaN,report1_00606M.pdf
...,...,...,...,...,...,...,...,...
91614B_02_12_2020,02_12_2020,91614B,Kildare,Celbridge Community School Moortown Maynooth R...,Post Primary,Whole School Evaluation,NaN,91614B_WSEMLLPP_14535_20200324.pdf
ST0008_30_06_2011,30_06_2011,ST0008,Dublin,Cara Park Traveller Training Centre Belcamp L...,Post Primary,Whole School Evaluation,NaN,report1_ST0008.pdf
YR0007_09_06_2011,09_06_2011,YR0007,Clare,Kilrush Youthreach Centre Adult Learning Centr...,Post Primary,Whole School Evaluation,NaN,report1_YR0007.pdf


**Now we can join the two together and drop the 'NaN'**
I do not understand why there are more rows (2026) than in data_df (2009)... 

In [84]:
general_inspectionReports.join(data_df, how='outer').dropna(subset=['sports_value'])

,Date,School Roll No.,County,School Name,School Level,Inspection Type,Subject,URL,sports_value
Report Id,,,,,,,,,
00467B_15_12_2011,15_12_2011,00467B,Cork,Ballinspittle National School Kinsale Cork,Primary,Whole School Evaluation,NaN,report1_00467B.pdf,1.0
00538V_25_10_2012,25_10_2012,00538V,Kerry,"Convent Primary School Daingean Uí Chúis, Co. ...",Primary,Whole School Evaluation,NaN,report1_EV_00538V.pdf,2.0
00606M_25_05_2015,25_05_2015,00606M,Tipperary N.R.,"Monard National School, Monard, Co Tipperary",Primary,Whole School Evaluation,NaN,report1_00606M.pdf,1.0
00729F_23_06_2014,23_06_2014,00729F,Dublin,"Loreto Primary School, Rathfarnham, Dublin 14",Primary,Whole School Evaluation,NaN,report1_00729F.pdf,0.0
00752A_15_10_2012,15_10_2012,00752A,Dublin,"Central Model Senior National School, Marlboro...",Primary,Whole School Evaluation,NaN,report1_00752A.pdf,0.0
...,...,...,...,...,...,...,...,...,...
91516B_21_09_2018,21_09_2018,91516B,Cork,"Skibbereen Community School, Main St, Skibbere...",Post Primary,Whole School Evaluation,NaN,91516B_WSEMLLPP_7225_20180516.pdf,0.0
91517D_24_06_2014,24_06_2014,91517D,Meath,"Athboy Community School, Athboy, Co. Meath",Post Primary,Whole School Evaluation,NaN,report7_91517D.pdf,1.0
91530S_07_11_2014,07_11_2014,91530S,Kildare,"Kildare Town Community School, Kildare, Co. Ki...",Post Primary,Whole School Evaluation,NaN,report3_91539S.pdf,0.0


# This notebook is over...
But in other notebooks, you can see **other topic-related word collections:**
- Positive and negative collections
- Topic-related words collections
**and how we examined them with pandas:**
- Exploring topics